### Exact Search

In [68]:
# import requests
# from dotenv import load_dotenv
# import os

# load_dotenv()
# API_KEY = os.getenv("COINDESK_API")  # Make sure this is in your .env file

# if not API_KEY:
#     raise ValueError("Please set COINDESK_API in your .env file")

# response = requests.get(
#     'https://data-api.coindesk.com/news/v1/search',  # ← No trailing spaces!
#     params={
#         "search_string": "bitcoin",
#         "lang": "EN",
#         "source_key": "coindesk",
#         "api_key": API_KEY,
#         "limit": 25,
#     },
#     headers={
#         "Content-Type": "application/json; charset=UTF-8"
#     }
# )

# if response.status_code != 200:
#     print("Status Code:", response.status_code)
#     print("Response Body:", response.text)
#     response.raise_for_status()

# json_response = response.json()
# articles = json_response.get("Data", [])
# print(f"Retrieved {len(articles)} articles.")

In [69]:
# articles

### Latest Crypto News Based on filter

In [74]:
import os
import requests
from dotenv import load_dotenv
from datetime import datetime
import pytz  # or use from zoneinfo import ZoneInfo if you prefer

load_dotenv()

COINDESK_API = os.getenv("COINDESK_API")

url = "https://data-api.coindesk.com/news/v1/article/list"
params = {
    "lang": "EN",
    "limit": 50,
    "sort": "published_at:desc"
}
headers = {
    "accept": "application/json",
    "x-api-key": COINDESK_API
}

response = requests.get(url, headers=headers, params=params)
response.raise_for_status()
coindesk_data = response.json()

BTC_KEYWORDS = ["bitcoin", "btc", "btc-usd", "btcusd", "btc-usdt", "btc price"]

def filter_btc_news(news_data):
    filtered = []
    for item in news_data.get("Data", []):
        combined_text = " ".join([
            item.get("TITLE", ""),
            item.get("BODY", ""),
            item.get("KEYWORDS", "")
        ]).lower()

        if any(keyword in combined_text for keyword in BTC_KEYWORDS):
            # Convert UNIX timestamp to Singapore Time
            try:
                utc_dt = datetime.utcfromtimestamp(item.get("PUBLISHED_ON", 0))
                sg_time = utc_dt.replace(tzinfo=pytz.utc).astimezone(pytz.timezone("Asia/Singapore"))
                formatted_time = sg_time.strftime("%Y-%m-%d %H:%M:%S %Z")
            except Exception:
                formatted_time = "N/A"

            filtered.append({
                "title": item.get("TITLE"),
                "url": item.get("URL"),
                "image": item.get("IMAGE_URL"),
                "published_on": formatted_time,
                "source": item.get("SOURCE_DATA", {}).get("NAME"),
                "sentiment": item.get("SENTIMENT"),
            })
    return filtered

btc_coindesk_news = filter_btc_news(coindesk_data)

btc_coindesk_news = sorted(
    btc_coindesk_news, key=lambda x: x["published_on"], reverse=True
)[:25]

# Display results
for i, n in enumerate(btc_coindesk_news, 1):
    print(f"## {i} ---> {n['published_on']} - {n['title']}")
    print(f"Source: {n['source']} | URL: {n['url']}")
    print(n)
    print()


## 1 ---> 2025-10-12 04:39:31 +08 - 2025 Nobel Peace Prize Winner María Corina Machado May See Bitcoin as Resistance Money for Venezuela
Source: CoinOtag | URL: https://en.coinotag.com/2025-nobel-peace-prize-winner-maria-corina-machado-may-see-bitcoin-as-resistance-money-for-venezuela/
{'title': '2025 Nobel Peace Prize Winner María Corina Machado May See Bitcoin as Resistance Money for Venezuela', 'url': 'https://en.coinotag.com/2025-nobel-peace-prize-winner-maria-corina-machado-may-see-bitcoin-as-resistance-money-for-venezuela/', 'image': 'https://resources.cryptocompare.com/news/77/53005695.jpeg', 'published_on': '2025-10-12 04:39:31 +08', 'source': 'CoinOtag', 'sentiment': 'POSITIVE'}

## 2 ---> 2025-10-12 04:31:04 +08 - The Latest Nobel Peace Prize Winner Is a Bitcoin Supporter
Source: Decrypt | URL: https://decrypt.co/343891/latest-nobel-peace-prize-winner-bitcoin-supporter
{'title': 'The Latest Nobel Peace Prize Winner Is a Bitcoin Supporter', 'url': 'https://decrypt.co/343891/la

C:\Users\leeee\AppData\Local\Temp\ipykernel_40588\2982123771.py:40: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  utc_dt = datetime.utcfromtimestamp(item.get("PUBLISHED_ON", 0))


RuntimeError: asyncio.run() cannot be called from a running event loop